## Dateien laden

In [1]:
import pandas as pd
import requests
from io import BytesIO

def load_csv_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return pd.read_csv(BytesIO(response.content))
    else:
        raise Exception(f"Download fehlgeschlagen: {url} – Status: {response.status_code}")

# SwitchDrive Download-Links
url_imdb_tmdb   = "https://drive.switch.ch/index.php/s/GknMWjEvz9VhuN4/download"
url_tmdb_credits = "https://drive.switch.ch/index.php/s/j36PM3I1C0FaX3C/download"
url_tmdb_movies  = "https://drive.switch.ch/index.php/s/SgdbbF6MkF0fTly/download"

# Lade die CSVs
df_imdb_tmdb   = load_csv_from_url(url_imdb_tmdb)
df_credits     = load_csv_from_url(url_tmdb_credits)
df_movies      = load_csv_from_url(url_tmdb_movies)

# Übersicht
print("IMDB+TMDB Combined:", df_imdb_tmdb.shape)
print("TMDB Credits:", df_credits.shape)
print("TMDB Movies:", df_movies.shape)


IMDB+TMDB Combined: (1072255, 42)
TMDB Credits: (4803, 4)
TMDB Movies: (4803, 20)


## Merge

In [2]:
# Sicherstellen, dass die IDs als Integer vorliegen
df_movies['id'] = df_movies['id'].astype(int)
df_credits['movie_id'] = df_credits['movie_id'].astype(int)

# Merge über TMDB Movie ID
df_tmdb_merged = pd.merge(df_movies, df_credits, how='inner', left_on='id', right_on='movie_id')

print("TMDB Combined:", df_tmdb_merged.shape)


TMDB Combined: (4803, 24)


In [3]:
print(df_tmdb_merged.columns)
print(df_imdb_tmdb.columns)


Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')
Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'production_companies',
       'production_countries', 'spoken_languages', 'keywords', 'release_year',
       'Director', 'AverageRating', 'Poster_Link', 'Certificate',
       'IMDB_Rating', 'Meta_score', 'Star1', 'Star2', 'Star3', 'Star4',
       'Writer', 'Director_of_Photography', 'Producers', 'Music_Composer',
       'genres_list', 'Cast_list'

In [4]:
# Auch hier sicherstellen, dass IDs Integer sind
df_tmdb_merged['id'] = df_tmdb_merged['id'].astype(int)
df_imdb_tmdb['id'] = df_imdb_tmdb['id'].astype(int)

# Merge über TMDB id
df_final = pd.merge(df_tmdb_merged, df_imdb_tmdb, how='inner', on='id')
print("Final Shape:", df_final.shape)

Final Shape: (4796, 65)


In [5]:
print(df_final.columns.tolist())


['budget_x', 'genres', 'homepage_x', 'id', 'keywords_x', 'original_language_x', 'original_title_x', 'overview_x', 'popularity_x', 'production_companies_x', 'production_countries_x', 'release_date_x', 'revenue_x', 'runtime_x', 'spoken_languages_x', 'status_x', 'tagline_x', 'title_x', 'vote_average_x', 'vote_count_x', 'movie_id', 'title_y', 'cast', 'crew', 'title', 'vote_average_y', 'vote_count_y', 'status_y', 'release_date_y', 'revenue_y', 'runtime_y', 'adult', 'backdrop_path', 'budget_y', 'homepage_y', 'imdb_id', 'original_language_y', 'original_title_y', 'overview_y', 'popularity_y', 'poster_path', 'tagline_y', 'production_companies_y', 'production_countries_y', 'spoken_languages_y', 'keywords_y', 'release_year', 'Director', 'AverageRating', 'Poster_Link', 'Certificate', 'IMDB_Rating', 'Meta_score', 'Star1', 'Star2', 'Star3', 'Star4', 'Writer', 'Director_of_Photography', 'Producers', 'Music_Composer', 'genres_list', 'Cast_list', 'overview_sentiment', 'all_combined_keywords']


In [8]:
# Alle Spalten mit _y löschen
df_final = df_final.drop(columns=[col for col in df_final.columns if col.endswith('_y')])

# Alle Spalten mit _x in umbenennen
df_final = df_final.rename(columns={col: col.replace('_x', '') for col in df_final.columns if col.endswith('_x')})


In [10]:
print("Spaltenübersicht nach _y/_x Bereinigung:")
print(df_final.columns.tolist())


Spaltenübersicht nach _y/_x Bereinigung:
['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'movie_id', 'cast', 'crew', 'title', 'adult', 'backdrop_path', 'imdb_id', 'poster_path', 'release_year', 'Director', 'AverageRating', 'Poster_Link', 'Certificate', 'IMDB_Rating', 'Meta_score', 'Star1', 'Star2', 'Star3', 'Star4', 'Writer', 'Director_of_Photography', 'Producers', 'Music_Composer', 'genres_list', 'Cast_list', 'overview_sentiment', 'all_combined_keywords']


In [11]:
# Nur Filme mit gültigem Umsatz und Budget
df_final_clean = df_final[(df_final['revenue'] > 0) & (df_final['budget'] > 0)].copy()

# Umbenennen für Klarheit
df_final_clean = df_final_clean.rename(columns={
    'revenue': 'revenue',
    'budget': 'budget'
})

print("Shape nach Bereinigung:", df_final_clean.shape)


Shape nach Bereinigung: (3229, 47)
